In [1]:
from pyplasm import*

Evaluating fenvs.py..
...fenvs.py imported in 0.005 seconds


##  Generazione di Tetti

Con questa semplificazione usata da alcuni colleghi ho scritto questo codice che genera un tetto su x,y,z dati in input le coridnate di un tetto sul piano x,y. La semplificazione sta nel fatto di conoscere a priori le bisettrici di ogni angolo e di dare indicazioni alla funzione su come calcolare i punti della parte alta del tetto dicendogli in quale quadrante passano le bisettrici degli angoli della figura.

### Altre idee per la realizazione 

Non soddisfatto appieno da quasta soluzione ho trovato su internet un algoritmo che permette di fare lo stesso lavoro utilizando un approccio un po piu geometrico, che si basa sullo studio delle bisettrici degli angoli della figura. Non sono ancora riuscito a riscriverlo al meglio ed a venire a capo di alcuni sotto-task geometrici; appena vi riuscirò la soluzione sarà reperibile in questa directory nel notebook chiamato workshop_09_v2

In [2]:
"""findLati
funzione ausiliare per la crezione di un dizionario dei lati della struttura

@param verts: i vertici della figura di cui si vogliono sapere i lati
@return: dizionario dei lati della figura
"""
def findLati(verts):
    lati={}
    i=0
    while(i<(len(verts)-1)):
        k=i+1
        lato=[verts[i],verts[i+1]]
        lati[k]=lato
        i=i+1
    return lati

"""create_ceiling
funzione per la creazione del soffito: prende i vertici della figura ed 
in funzione dell'angolo, si trova un nuovo punto x,y,z calcolato supponendo che l'ipotenusa sia 1 
e con l'angolo passato come parametro NB:degree rappresenta i gradi per cui passa la bisettrice dell'angolo

@param verts: i vertici della figura di cui si vogliono sapere i lati
@param rotation: su che asse si trova l'angolo della figura
@param degree:  gradi per cui passano la bisettrici degli angoli
@return: lista dei punti che fanno da soffitto
""" 
def create_ceiling(verts,rotation,degree):
    i=0
    ceiling=[]
    for i in range(len(rotation)):
        ceiling_p = []
        if(rotation[i]==1): 
            newX=verts[i][0]+math.cos(degree[i])
            newY= verts[i][1] +math.sin(degree[i])
            ceiling_p=[newX,newY,1]

        if(rotation[i]==2): 
            newX=verts[i][0] -1*math.cos(degree[i])
            newY= verts[i][1]+ math.sin(degree[i])
            ceiling_p=[newX,newY,1]     

        if(rotation[i]==3): 
            newX=verts[i][0] -1* math.cos(degree[i])
            newY= verts[i][1] -1*math.sin(degree[i])
            ceiling_p=[newX,newY,1]

        if(rotation[i]==4): 
            newX=verts[i][0]+ math.cos(degree[i])
            newY= verts[i][1]-1*math.sin(degree[i])
            ceiling_p=[newX,newY,1]  

        ceiling.append(ceiling_p)
    
    
    return ceiling

"""build_planes
crea i pannelli che formano il soffitto, creando dei parallelogrammi sulla base
del contenuto delle mappe rappresentanti i lati della base ed i lati del soffitto

@param verts: i vertici della figura
@param verts: i vertici del soffitto della figura
@return: hpc dei pannelli laterali del tetto
"""

def build_planes(verts,points_landing):
    baseLati=findLati(verts)
    points_landing.append(points_landing[0])
    roofLati=findLati(points_landing)

    roof=[]

    for i in (baseLati.keys()):
        points=[]
        points.append(baseLati[i][0])
        points.append(roofLati[i][0])
        points.append(roofLati[i][1])
        points.append(baseLati[i][1])
        plane = MKPOL([points,[[4,3,2,1]],1])
        roof.append(plane)
    
    return (STRUCT(roof))


In [3]:
"""create_my_roof
funzione principale che si occupa di creare il tetto

@param verts: i vertici della figura
@param cells: celle
@param rotation: lista di assi su cui trova l'angolo
@param radiant: l'angolo per cui passa la bisettrice 
@return: hpc del tetto
"""

def create_my_roof(verts,cells,rotation,radiant):
    points_landing=create_ceiling(verts,rotation,radiant)
    landing = (MKPOL([points_landing,cells,1]))  
    planes = build_planes(verts,points_landing)
    final_roof= STRUCT([planes,landing])
    final_roof=TEXTURE(["./pattern/tegole.jpg", TRUE, TRUE, 5, 5, 0, 5, 5])(final_roof)
    VIEW(final_roof)
    return final_roof

In [4]:
"""tetto semplice"""
verts = [[0,0,0],[2,0,0],[2,2,0],[0,2,0],[0,0,0]]
cells = [[1,2,3,4]]
rotation=[1,2,3,4]
radiant=[PI/4,PI/4,PI/4,PI/4,PI/4]
create_my_roof(verts,cells,rotation,radiant)
"""tetto a C capovolta"""
verts = [[0,0,0],[8,0,0],[8,4,0],[5,4,0],[5,2,0],[3,2,0],[3,4,0],[0,4,0],[0,0,0]]
cells = [[1,2,5,6],[2,3,4,5],[6,7,8,1]]
rotation=[1,2,3,4,4,3,3,4]
radiant=[PI/4,PI/4,PI/4,PI/4,PI/4,PI/4,PI/4,PI/4]
create_my_roof(verts,cells,rotation,radiant)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x105765ae0> >

### Risultati

Tetto semplice

![prima immagine](https://github.com/adixia/ggpl-LAB/blob/master/2016-12-16/img/img4.png?raw=true)

![prima immagine](https://github.com/adixia/ggpl-LAB/blob/master/2016-12-16/img/img5.png?raw=true)

Tetto a forma di C capovolta 

![prima immagine](https://github.com/adixia/ggpl-LAB/blob/master/2016-12-16/img/img1.png?raw=true)

![prima immagine](https://github.com/adixia/ggpl-LAB/blob/master/2016-12-16/img/img2.png?raw=true)

![prima immagine](https://github.com/adixia/ggpl-LAB/blob/master/2016-12-16/img/img3.png?raw=true)
